In [19]:
import os
import numpy as np
import tensorflow as tf
from flask import Flask, request, jsonify
from PIL import Image
import io
from threading import Thread

# Flask uygulamasını başlatın
app = Flask(__name__)

# Eğittiğiniz modeli yükleyin
model = tf.keras.models.load_model('unet_model.h5')

# Modeli derleyin
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Görüntü boyutunu modele uygun olacak şekilde ayarlayın (örneğin 128x128)
IMAGE_SIZE = (128, 128)

# Görüntüleri işlemek için bir yardımcı fonksiyon
def preprocess_image(image):
    image = image.resize(IMAGE_SIZE)  # Görüntüyü boyutlandırın
    image = np.array(image) / 255.0   # Normalizasyon
    image = np.expand_dims(image, axis=0)  # Modelin kabul edebileceği şekle getirin
    return image

# Ana API route'u: Görüntü gönderilip maske döndürülen kısım
@app.route('/predict', methods=['POST'])
def predict():
    # İstekten dosyayı alın
    if 'image' not in request.files:
        return jsonify({'error': 'Görüntü dosyası bulunamadı'}), 400

    file = request.files['image']

    # Dosyayı bir PIL görüntüsüne çevirin
    try:
        image = Image.open(file.stream).convert("RGB")
    except:
        return jsonify({'error': 'Geçersiz görüntü formatı'}), 400

    # Görüntüyü ön işleme tabi tutun
    preprocessed_image = preprocess_image(image)

    # Modelden tahmin alın
    prediction = model.predict(preprocessed_image)

    # Tahminleri ikili maske haline getirin (0 veya 255 değerlerinde olacak)
    mask = (prediction[0] > 0.5).astype(np.uint8) * 255

    # Hasar analizi: Maskede 255 değerinde (hasarlı alanlar) kaç piksel olduğunu say
    damage_ratio = np.sum(mask == 255) / mask.size  # Hasarlı piksellerin oranı

    # Hasar oranı %10'dan fazla ise "hasarlı", aksi takdirde "hasarsız" olarak işaretle
    threshold = 0.1  # %10 eşik
    if damage_ratio > threshold:
        result = "hasarlı"
    else:
        result = "hasarsız"

    # Sonucu JSON formatında döndür
    return jsonify({'result': result})

# Sunucuyu Colab'de arka planda başlatmak için thread kullanın
def run():
    app.run(host='0.0.0.0', port=5000)

# Flask uygulamasını ayrı bir thread'de başlatın
thread = Thread(target=run)
thread.start()
